In [ ]:
from bs4 import BeautifulSoup
import requests
from os.path import join, basename, splitext
import re
import json
from tqdm import tqdm

In [ ]:
url = "https://www.musik-produktiv.de/e-gitarre/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')


In [ ]:
soup.find("div", class_="list_page").find_all("a")[-1].text

In [ ]:
def download_image(guitar_url,data_id, headers):
    ext = splitext(basename(guitar_url))[-1]
    filepath = join("guitar_images",data_id + ext)

    img_data = requests.get(url=guitar_url, headers=headers).content
    with open(filepath, 'wb') as handler:
        handler.write(img_data)

def store_attributes(page, data_id):
    attribute_list = page.find("ul", class_="eigenschaften")
    attributes = [x.text for x in attribute_list.find_all("li")]
    filepath = join("guitar_attributes",data_id + '.txt')
    with open(filepath, 'w') as file:
        [file.write(x+'\n') for x in attributes]

def store_description(page, data_id):
    description = page.find("div", class_="product-description")
    filepath = join("guitar_description",data_id + '.txt')
    with open(filepath, 'w') as file:
        file.write(description)

def get_id(g_url):
    return g_url.split('/')[-1].replace(".html", "")

def store_reviews(page, data_id):
    reviews = page.find_all("div", class_="review")
    out = [read_review(x) for x in reviews]
    with open(join("guitar_reviews",data_id+'.json'), 'w') as file:
        json.dump(out,file)

def read_review(rev_item):
    out = {}
    rgx = 'width: (\d+)%'
    _, keys_, title, user, text, _ = rev_item.text.split('\n')

    out.update({
        "title":title,
        "text":text
    })

    stars_ = rev_item.find_all("span",class_="mp-stars")
    # "width: 100%" -> 100 -> 100/20 -> 5 stars
    keys_ = [x for x in keys_.split(" ") if x != ""]
    assert len(keys_) == len(stars_)
    for key,star in zip(keys_,stars_):
        num_stars = int(re.match(rgx, star.find("span")['style']).group(1))/20
        out[key] = num_stars

    return out

In [ ]:
grid = soup.find("ul", class_="artgrid clearfix col-4")
guitar_urls = [ x.a['href'] for x in grid.find_all("li")]

In [ ]:
for g_url in tqdm(guitar_urls):
    data_id = get_id(g_url)
    tmp_page = requests.get(g_url, data_id, headers=headers)
    tmp_page = BeautifulSoup(tmp_page.content, 'html.parser')
    image_url = tmp_page.find(id="zoom")['href']
    download_image(image_url, data_id, headers)
    store_attributes(tmp_page, data_id)
    store_reviews(tmp_page, data_id)
    break
    